# CLEANING DATA

### Importing libraries

In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Load the dataset

In [4]:
data = pd.read_csv('Cleaning data.csv')

In [5]:
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


### 1. **Data Integrity**: Ensuring the accuracy, consistency, and reliability of data

In [6]:
print("Dataset Overview:\n")
print("Initial Data Shape:", data.shape)
print("Column Data Types:\n", data.dtypes)
print("Summary Statistics:\n", data.describe(include='all'))

Dataset Overview:

Initial Data Shape: (48895, 16)
Column Data Types:
 id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object
Summary Statistics:
                   id            name       host_id host_name  \
count   4.889500e+04           48879  4.889500e+04     48874   
unique           NaN           47905           NaN     11452   
top              NaN  Hillside

### 2. **Missing Data Handling**

In [7]:
print("\nMissing Values Before Handling:")
print(data.isnull().sum())


Missing Values Before Handling:
id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [9]:
# Handling missing values:
# Numerical columns: Fill with mean
data.fillna(data.mean(numeric_only=True), inplace=True)

# Categorical columns: Fill with mode
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [10]:
print("\nMissing Values After Handling:")
print(data.isnull().sum())


Missing Values After Handling:
id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64


### 3. **Duplicate Removal**

In [11]:
print("\nDuplicate Rows Before Removal:", data.duplicated().sum())
data.drop_duplicates(inplace=True)
print("Duplicate Rows After Removal:", data.duplicated().sum())


Duplicate Rows Before Removal: 0
Duplicate Rows After Removal: 0


### 4. **Standardization**: Consistent formatting and units

In [12]:
# Standardize column names: lowercase, strip spaces, and replace spaces with underscores
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_')
print("\nStandardized Column Names:")
print(data.columns)


Standardized Column Names:
Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


### 5. **Outlier Detection**

In [13]:
# Detecting outliers using the IQR method
def detect_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col] < lower_bound) | (df[col] > upper_bound)]

# Identify outliers for all numerical columns
numerical_cols = data.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    outliers = detect_outliers(data, col)
    print(f"\nOutliers in '{col}': {len(outliers)} records detected")


Outliers in 'id': 0 records detected

Outliers in 'host_id': 1526 records detected

Outliers in 'latitude': 425 records detected

Outliers in 'longitude': 2833 records detected

Outliers in 'price': 2972 records detected

Outliers in 'minimum_nights': 6607 records detected

Outliers in 'number_of_reviews': 6021 records detected

Outliers in 'reviews_per_month': 4103 records detected

Outliers in 'calculated_host_listings_count': 7081 records detected

Outliers in 'availability_365': 0 records detected


In [19]:
#remove outliers
for col in numerical_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]

### Final Dataset Overview

In [16]:
print("\nCleaned Data Shape:", data.shape)
print("Cleaned Data Preview:\n", data.head())


Cleaned Data Shape: (23268, 16)
Cleaned Data Preview:
        id                                               name  host_id  \
10   5295                   Beautiful 1br on Upper West Side     7702   
20   7801                   Sweet and Spacious Brooklyn Loft    21207   
25   8505                 Sunny Bedroom Across Prospect Park    25326   
26   8700  Magnifique Suite au N de Manhattan - vue Cloitres    26394   
48  13050                          bright and stylish duplex    50846   

          host_name neighbourhood_group       neighbourhood  latitude  \
10             Lena           Manhattan     Upper West Side  40.80316   
20            Chaya            Brooklyn        Williamsburg  40.71842   
25          Gregory            Brooklyn     Windsor Terrace  40.65599   
26  Claude & Sophie           Manhattan              Inwood  40.86754   
48         Jennifer            Brooklyn  Bedford-Stuyvesant  40.68554   

    longitude        room_type  price  minimum_nights  number_of_r